# Wstęp
Zadanie 10 jest pierwszą częścią zajęć laboratoryjnych poświęconych sieciom rekurencyjnym i predykcji z wykorzystaniem danych multimodalnych. Efektem prac będzie sieć rekurencyjna do predykcji kursu kryptowaluty Bitcoin (BTC) w oparciu o dane z giełdy oraz o wyniki analizy emocji komunikatów z mediów społecznościowych, do których również należy utworzyć dedykowany model sieci rekurencyjnej. Plan realizacji etapów wygląda następująco:

1.   EmoTweet - model sieci rekurencyjnej do analizy emocji (10 pkt., laboratorium 10)
2. Agregacja informacji emotywnej i przygotowanie MultiBTC - multimodalnego model sieci rekurencyjnej do predykcji kursu BTC (10 pkt., laboratorium 11)
3. Ewaluacja modelu MultiBTC (10 pkt., laboratorium 12)

Łącznie można otrzymać 30 punktów.

# Cel ćwiczenia

Celem pierwszego etapu prac jest zapoznanie się z podstawową siecią rekurencyjną LSTM. Ze względu na fakt, że model ten będzie wykorzystany do analizy emocji tekstu, w ramach teorii do zadania zostanie omówiony podstawowy mechanizm konwersji słów w tekście do postaci wektorów dystrybucyjnych (tzw. word embeddings) na podstawie rozwiązania o nazwie `fastText`. Modele będą budowane na ogólnodostępnym zbiorze `TweetEval`, zawierającym podzbiory ręcznie anotowanych tweetów przy pomocy etykiet odnoszących się do następujących zjawisk: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

# Warunki zaliczenia

Do zaliczenia pierwszego etapu należy utworzyć następujące modele dla min. 2 wybranych zjawisk:

1.   Model bazowy (regresja logistyczna).
2.   Model rekurencyjny oparty o sieć LSTM.

Wytrenowane modele będą wykorzystane w 2 etapie, dlatego proszę je zachować.

# Wektory dystrybucyjne

W przetwarzaniu języka naturalnego, o wektorach dystrybucyjnych (inaczej osadzeniach lub zanurzeniach, ang. word embeddings) mówi się w kontekście reprezentacji słów w tekście, zazwyczaj w postaci wektora liczb rzeczywistych, który koduje znaczenie słowa. Hipoteza dystrybucyjna, u podstawy której leży większość metod reprezentacji, mówi o tym, że słowa, które często współwystępują, mają podobne znaczenie. Wektory dystrybucyjne można uzyskać za pomocą zestawu technik modelowania języka, w których słowa lub frazy są mapowane do wektorów liczb rzeczywistych. Z reguły polega to na matematycznym zanurzeniu z przestrzeni o wielu wymiarach opisujących słowo (konteksty) do ciągłej przestrzeni wektorowej o znacznie mniejszym wymiarze.

Metody generowania tego odwzorowania obejmują sieci neuronowe, redukcję wymiarowości na macierzy współwystępowania słów, modele probabilistyczne lub jawną reprezentację w kontekście, w którym pojawiają się słowa. Wektory dystrybucyjne, używane jako podstawowa reprezentacja wejściowa tekstu, okazały się istotnie poprawiać jakość w wielu zadaniach NLP, takich jak np. rozpoznawanie nazw własnych, określanie części mowy, rozpoznawanie dziedziny tekstu, czy też rozpoznawanie wydźwięku i emocji w tekście. 

# fastText

[fastText](https://fasttext.cc/) jest biblioteką do efektywnego uczenia modeli reprezentacji wektorowych słów oraz do budowania klasyfikatorów tekstu. Modele językowe można budować z wykorzystaniem dwóch popularnych technik: [Continuous Bag of Words](https://www.kdnuggets.com/2018/04/implementing-deep-learning-methods-feature-engineering-text-data-cbow.html) oraz [Skip-Gram](https://towardsdatascience.com/skip-gram-nlp-context-words-prediction-algorithm-5bbf34f84e0c). 

## Instalacja

Pobranie repozytorium projektu:


In [4]:
!git clone https://github.com/facebookresearch/fastText.git

fatal: destination path 'fastText' already exists and is not an empty directory.


Instalacja biblioteki:

In [5]:
!cd fastText && mkdir build && cd build && cmake ..  && make && make install

mkdir: cannot create directory ‘build’: File exists


Instalacja API do Pythona:

In [1]:
!cd fastText && pip install .

Processing /app/fastText
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4211752 sha256=23a4ee5b83bbf5b7c8d3c91cf93b7e8fe3fb55460295183d490dc1de94a3b8fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-tkgw_mak/wheels/34/9a/79/ffb29de213ba6929d4e91e57b406e6f9ba1b7eba23dfa5c47d
Successfully built fasttext


In [2]:
import fasttext

help(fasttext.FastText)


Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You might want to consult standard preprocessi

In [143]:
import torch

if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!


# Dane do etapu nr 1

## Korpus 
Korpus (zbiór dokumentów) do realizacji etapu nr 1 pochodzi z repozytorium [TweetEval](https://github.com/cardiffnlp/tweeteval). Repozytorium zawiera 7 różnorodnych zbiorów danych, zawierających zanonimizowane wpisy z [Twittera](https://twitter.com), anotowane następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance). 

In [8]:
import os

folder_path = "/dane"
if os.path.exists(folder_path):
    print("The 'dane' folder is visible.")
else:
    print("The 'dane' folder is not visible.")

The 'dane' folder is visible.


In [8]:
!7za x dane/tweeteval.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,8 CPUs 13th Gen Intel(R) Core(TM) i5-13400F (B06F2),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 17390348 bytes (17 MiB)

Extracting archive: dane/tweeteval.7z
--
Path = dane/tweeteval.7z
Type = 7z
Physical Size = 17390348
Headers Size = 1810
Method = LZMA2:24
Solid = +
Blocks = 1

      0% 27 - tweeteval/datasets/stance/atheis                                           10% 40 - tweeteval/.git/hooks/pre-applypatch.samp                                                   10% 51 - tweeteval/.git/logs/refs/heads/mai                                             39% 55 - tweeteval/.git/packed-re                                   49% 64 - tweeteval/datasets/emoji/train_labels.tx                                                   59% 68 - tweeteval/datasets/emotion/mapping.t                                               59% 78 - tweetev

## Zawartość korpusu

W katalogu głównym (tweeteval) znajdują się następujące elementy:
*   `datasets` - katalog ze zbiorami danych
   * `emotion` - tweety anotowane emocjami 
     * `mapping.txt` - identyfikatory etykiet oraz ich opis
     * `train_text.txt` - wpisy z Twittera (część ucząca)
     * `train_labels.txt` - etykiety wpisów z Twittera (część ucząca)
     * `test_*.txt, valid_*.txt` - j.w. (część testowa i walidacyjna)
   * `emoji` - tweety anotowane emotikonami
   * `...` - katalogi zawierające tweety anotowane pozostałymi zjawiskami
*   `predictions` - katalog z przykładowymi predykcjami
   * `emotion.txt` - etykiety modelu predykcyjnego dla części testowej danych `emotion`
   * `emoji.txt` - j.w. dla cz. testowej danych `emoji`
   * `...` - j.w. dla pozostałych danych
*   `evaluation_script.py` - skrypt do ewaluacji 

## Model języka

Na potrzeby zadania został przygotowany model Skip-Gram reprezentacji wektorowej słów, zbudowany na wielkim korpusie tweetów dotyczących kursu BTC. Wersja binarna tego modelu dostępna jest w 2 wariantach:
* wektory 100-elementowe (1.7GB, fasttext_tweetmodel_btc_sg_100_en.bin)
* wektory 20-elementowe (350MB, fasttext_tweetmodel_btc_sg_20_en.bin)

Na potrzeby prezentacji przykładowego rozwiązania zostanie wykorzystany mniejszy model. Do realizacji ostatecznego rozwiązania należy wykorzystać większy model. 



# Model bazowy rozpoznawania emocji

Model bazowy, zbudowany z wykorzystaniem narzędzia fastText (oparty o regresję logistyczną), będzie punktem wyjścia do porównania się z modelami opartymi o sieci LSTM, których skonstruowanie i ewaluacja na wybranych zadaniach będzie celem etapu nr 1. 

Pobranie mniejszego modelu reprezentacji języka tweetów:


In [ ]:
# należy wgrać plik z katalogu "dane" o nazwie fasttext_tweetmodel_btc_sg_20_en.bin

Wydobycie słownika wektorów z binarnego modelu języka:

In [17]:
!ls /usr/bin/python*

/usr/bin/python3	 /usr/bin/python3.10
/usr/bin/python3-config  /usr/bin/python3.10-config


In [18]:
!/usr/bin/python3 fastText/python/doc/examples/bin_to_vec.py dane/fasttext_tweetmodel_btc_sg_20_en.bin > dane/fasttext_tweetmodel_btc_sg_20_en.vec

Dodanie prefiksu `__label__` do etykiet zbioru `emotion`:

In [19]:
!sed 's/^/__label__/g' tweeteval/datasets/emotion/train_labels.txt > train_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/test_labels.txt > test_labels_emo.txt
!sed 's/^/__label__/g' tweeteval/datasets/emotion/val_labels.txt > val_labels_emo.txt

Przygotowanie zbioru uczącego, testowego i walidacyjnego w formacie `fastText`:

In [20]:
!paste -d " " tweeteval/datasets/emotion/train_text.txt train_labels_emo.txt > train_emo.txt
!paste -d " " tweeteval/datasets/emotion/test_text.txt test_labels_emo.txt > test_emo.txt
!paste -d " " tweeteval/datasets/emotion/val_text.txt val_labels_emo.txt > val_emo.txt

Trenowanie modelu z wykorzystaniem wejścia `train_emo.txt`, z określeniem wyjściowej nazwy modelu `emo_model`, dla wektorów słów o wymiarze `20`, z wykorzystaniem pretrenowanych wektorów z pliku `fasttext_tweetmodel_btc_sg_20_en.vec` i z uruchomieniem dostrajania hiperparametrów na zbiorze walidacyjnym `val_emo.txt`:

In [28]:
!find / -name fasttext

/home/jovyan/fastText/build/fasttext
/home/jovyan/fastText/build/lib.linux-x86_64-3.10/fasttext
/home/jovyan/fastText/build/temp.linux-x86_64-3.10/python/fasttext_module/fasttext
/home/jovyan/fastText/python/fasttext_module/fasttext
/usr/local/lib/python3.10/dist-packages/fasttext
/usr/local/bin/fasttext
/app/fastText/build/fasttext
/app/fastText/build/lib.linux-x86_64-3.10/fasttext
/app/fastText/build/temp.linux-x86_64-3.10/python/fasttext_module/fasttext
/app/fastText/python/fasttext_module/fasttext


In [29]:
!/home/jovyan/fastText/build/fasttext supervised -input train_emo.txt -output emo_model -dim 20 -pretrainedVectors dane/fasttext_tweetmodel_btc_sg_20_en.vec -autotune-validation val_emo.txt 

Progress: 100.0% Trials:   60 Best score:  0.692513 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  12887
Number of labels: 4
Progress: 100.0% words/sec/thread:  499013 lr:  0.000000 avg.loss:  0.572941 ETA:   0h 0m 0s


Podstawowa ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision) i kompletność (R - recall) w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [31]:
!/home/jovyan/fastText/build/fasttext test emo_model.bin test_emo.txt

N	1421
P@1	0.699
R@1	0.699


Rozszerzona ewaluacja modelu z wykorzystaniem `fastText`, wynikiem jest precyzja (P - precision), kompletność (R - recall) oraz F1-score dla każdej etykiety w wariancie [weighted](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html).

In [32]:
!/home/jovyan/fastText/build/fasttext test-label emo_model.bin test_emo.txt

F1-Score : 0.771104  Precision : 0.704748  Recall : 0.851254   __label__0
F1-Score : 0.682477  Precision : 0.687003  Recall : 0.678010   __label__3
F1-Score : 0.670769  Precision : 0.746575  Recall : 0.608939   __label__1
F1-Score : 0.407960  Precision : 0.525641  Recall : 0.333333   __label__2
N	1421
P@1	0.699
R@1	0.699


Przygotowanie danych do ewaluacji z wykorzystaniem skryptu dołączonego do zbioru TweetEval:

In [34]:
!mkdir predictions2

mkdir: cannot create directory ‘predictions2’: File exists


In [35]:
!/home/jovyan/fastText/build/fasttext predict emo_model.bin tweeteval/datasets/emotion/test_text.txt | sed 's/__label__//g' > predictions2/emotion.txt

Uruchomienie ewaluacji. Oprócz wyników P, R, F1 [weighted]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)) dla każdej etykiety, otrzymujemy również wyniki w wariancie [macro]((https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html)). **Ostateczną miarą (TweetEval Score) jest miara F1-score w wariancie macro i tę miarę proszę traktować jako kluczową przy porównywaniu rozwiązań.**

In [37]:
!/usr/bin/python3 tweeteval/evaluation_script.py --tweeteval_path tweeteval/datasets --predictions_path predictions2 --task emotion

0 {'precision': 0.7047477744807121, 'recall': 0.8512544802867383, 'f1-score': 0.771103896103896, 'support': 558}
1 {'precision': 0.7465753424657534, 'recall': 0.6089385474860335, 'f1-score': 0.6707692307692307, 'support': 358}
2 {'precision': 0.5256410256410257, 'recall': 0.3333333333333333, 'f1-score': 0.40796019900497504, 'support': 123}
3 {'precision': 0.6870026525198939, 'recall': 0.6780104712041884, 'f1-score': 0.6824769433465085, 'support': 382}
accuracy 0.6988036593947924
macro avg {'precision': 0.6659916987768463, 'recall': 0.6178842080775734, 'f1-score': 0.6330775673061526, 'support': 1421}
weighted avg {'precision': 0.6950120268679963, 'recall': 0.6988036593947924, 'f1-score': 0.6905676674717359, 'support': 1421}
------------------------------
TweetEval Score (emotion): 0.6330775673061526


# Budowa modeli EmoTweet

W tej sekcji Państwa zadaniem będzie przygotowanie modeli sieci LSTM oraz modeli bazowych opartych o regresję logistyczną (fastText) dla wybranych 2 zjawisk ze zbioru TweetEval. Dla sieci LSTM kolejne jednostki sieci rekurencyjnej na wejściu dostają reprezentację wektorową kolejnych wyrazów w tekście. Wyjście z ostatniej jednostki podlega klasyfikacji. W celu usprawnienia zadania, przedstawiona zostanie metoda reprezentacji wektorowej tekstu z wykorzystaniem Pythonowego API do narzędzia fastText. Do ewaluacji modeli należy wykorzystać uprzednio zaprezentowany skrypt `tweeteval/evaluation_script.py`.

## Wektoryzacja tekstu


In [4]:
# inicjalizacja biblioteki
import fasttext

In [5]:
# ładowanie modelu
MODEL_PATH = 'dane/fasttext_tweetmodel_btc_sg_20_en.bin'
model = fasttext.load_model(MODEL_PATH)

In [6]:
# wczytanie danych treningowych
import pandas as pd
TRAIN_PATH = 'tweeteval/datasets/emotion/train_text.txt'
train_texts = pd.read_csv(TRAIN_PATH, sep='\t', header=None)
train_texts

,0
0,“Worry is a down payment on a problem you may ...
1,My roommate: it's okay that we can't spell bec...
2,No but that's so cute. Atsu was probably shy a...
3,Rooneys fucking untouchable isn't he? Been fuc...
4,it's pretty depressing when u hit pan on ur fa...
...,...
3252,I get discouraged because I try for 5 fucking ...
3253,The @user are in contention and hosting @user ...
3254,@user @user @user @user @user as a fellow UP g...
3255,You have a #problem? Yes! Can you do #somethin...


In [42]:
# wektoryzacja pierwszego tekstu
first_text = train_texts[0][0]
for word in fasttext.tokenize(first_text):
  print(word, model.get_word_id(word), model.get_word_vector(word))

“Worry -1 [-0.04189867  0.15429688  0.96717507  1.3809655   0.49123076 -0.5447607
 -0.11276884  0.20356484 -1.0640966  -1.6616327   0.03930127 -0.7224096
  0.21334486 -0.5872285   0.2898182   0.81751084 -1.6077403   1.8038087
  0.4850348   1.0643197 ]
is 6 [ 0.24099417  0.13544752  0.7251924   0.32544732  0.27421224  0.31903243
  0.7501186   0.22853182 -0.91543657  0.08587569  0.13866538 -0.38624704
 -0.30637258  0.13666666 -0.43992838 -0.12443608 -1.0383893  -0.06567164
  0.17007533 -0.16708991]
a 7 [-0.00810981 -0.03934941  0.81658655  0.56301105  0.43812367  0.29547286
  0.4691784   0.07483605 -0.58705056  0.28240088 -0.6339584  -0.16187707
 -0.23376046 -0.1245347   0.03071329 -0.07603034 -0.9066614  -0.07007706
  0.4522892  -0.15033531]
down 174 [ 0.9175071  -1.0815151   0.07119758  0.34226617  0.9607946   0.5973182
  0.91058624 -0.32068744 -0.72137564  1.2241784  -0.1882128  -0.23591968
 -0.02596712 -0.10194965 -0.09553405  0.36303622  0.22354192  0.4901933
  0.5405883   0.5965071

Proszę zwrócić uwagę, że fastText jest w stanie przyporządkować reprezentację wektorową nawet dla takich słów, których model języka nie widział w trakcie uczenia (pierwszy token wejściowego tekstu). 

## Model klasyfikacji tekstu LSTM (2 pkt.)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class LSTMClassifier0(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim):
        super(LSTMClassifier0, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden = hidden[-1]
        output = self.fc(hidden)
        return output

In [3]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, lengths):
        packed_x = nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (hidden, _) = self.lstm(packed_x)
        output, _ = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        hidden = self.dropout(hidden[-1])
        output = self.fc(hidden)
        return nn.functional.softmax(output, dim=1)


In [4]:
class TextDataset0(Dataset):
    def __init__(self, texts, labels, model):
        self.model = model
        self.texts = [self.preprocess(text) for text in texts]
        self.labels = labels

    def preprocess(self, text):
        return [self.model.get_word_vector(word) for word in fasttext.tokenize(text)]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'text': torch.tensor(self.texts[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [5]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, model):
        self.model = model
        self.texts = [self.preprocess(text) for text in texts]
        self.labels = labels

    def preprocess(self, text):
        return [self.model.get_word_vector(word) for word in fasttext.tokenize(text)]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            'text': torch.tensor(self.texts[idx], dtype=torch.float),
            'label': torch.tensor(self.labels[idx], dtype=torch.long),
            'length': len(self.texts[idx])
        }

In [6]:
BASE_PATH = 'tweeteval/datasets'
FT_MODEL_PATH = 'dane/fasttext_tweetmodel_btc_sg_20_en.bin'

## Trenowanie modeli LSTM dla ZJAWISKO_1 i ZJAWISKO_2 (2 pkt.)
Należy wybrać 2 z 7 dostępnych podzbiorów z [TweetEval](https://github.com/cardiffnlp/tweeteval) anotowanych następującymi zjawiskami: 1) emocje (emotion), 2) emotikony (emoji), 3) ironia (irony), 4) mowa nienawiści (hate speech), 5) mowa ofensywna (offensive language), 6) wydźwięk (sentiment), 7) nastawienie (stance).

In [7]:
from tqdm import tqdm
import pandas as pd
from torch.nn.utils.rnn import pad_sequence


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def get_texts_and_labels(base_dataset_path, dataset_name, data_type="train"):
    base_path = f'{base_dataset_path}/{dataset_name}'
    text_path = f'{base_path}/{data_type}_text.txt'
    labels_path = f'{base_path}/{data_type}_labels.txt'
    
    train_texts = pd.read_csv(text_path, sep='\t', header=None, names=['text'])
    train_labels = pd.read_csv(labels_path, sep='\t', header=None, names=['label'])
    
    df = pd.concat([train_texts, train_labels], axis=1)
    df.dropna(inplace=True)
    
    texts = df['text'].tolist()
    labels = df['label'].tolist()
    
    return texts, labels


def collate_fn(batch):
    batch.sort(key=lambda x: x['length'], reverse=True)
    sequences, labels, lengths = zip(*[(item['text'], item['label'], item['length']) for item in batch])
    sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)
    return {'text': sequences, 'label': labels, 'length': lengths}


def get_dataloader(model_path, base_dataset_path, dataset_name, data_type="train"):
    model = fasttext.load_model(model_path)
    texts, labels = get_texts_and_labels(base_dataset_path, dataset_name, data_type)

    dataset = TextDataset(texts, labels, model)
    dataloader = DataLoader(dataset, batch_size=32, collate_fn=collate_fn)
    class_count = len(set(labels))

    return dataloader, model.get_dimension(), class_count

In [170]:
def train_lstm_model(emotion, num_epochs=100, hidden_dim=50, dropout_rate=0.7):
    dataloader, embedding_dim, class_count = get_dataloader(FT_MODEL_PATH, BASE_PATH, emotion)

    hidden_dim = hidden_dim # 100-500 => tuning
    output_dim = class_count
    lstm_model = LSTMClassifier(embedding_dim, hidden_dim, output_dim, dropout_rate=dropout_rate).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(lstm_model.parameters(), lr=1e-4, weight_decay=1e-5)
    print(embedding_dim, class_count)
    return
    for epoch in range(num_epochs):
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        for batch in loop:
            texts = batch['text'].to(device)
            labels = batch['label'].to(device)
            lengths = batch['length']

            outputs = lstm_model(texts, lengths)

            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loop.set_description(f"Epoch {epoch+1}/{num_epochs}")
            loop.set_postfix(loss=loss.item())
            
    return lstm_model


hate_lstm_model = train_lstm_model("hate")
offensive_lstm_model = train_lstm_model("offensive")

Epoch 6/100: 100%|██████████| 373/373 [00:04<00:00, 88.34it/s, loss=0.645]


Epoch 100/100: 100%|██████████| 373/373 [00:03<00:00, 118.32it/s, loss=0.722]


In [176]:
torch.save(hate_lstm_model.state_dict(), 'hate_lstm_model.pt')
torch.save(offensive_lstm_model.state_dict(), 'offensive_lstm_model.pt')

## Trenowanie modeli LR (fastText) dla ZJAWISKO_1 i ZJAWISKO_2 (2 pkt.)

In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np


def vectorize_text(text, model):
    if pd.isna(text):
        return np.zeros(model.get_dimension())

    word_vectors = [model.get_word_vector(word) for word in fasttext.tokenize(text)]
    return word_vectors


def get_avg_vector(texts, model):
    def text_to_avg_vector(text, model):
        word_vectors = vectorize_text(text, model)
        return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.get_dimension())

    avg_word_vectors = np.array([text_to_avg_vector(text, model) for text in texts])
    return avg_word_vectors


def scale(X_train, X_test):
    scaler = StandardScaler()

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled


def predict(
    classifier,
    X_Train,
    X_test,
    y_train,
):
    classifier.fit(X_Train, y_train)

    y_pred_train = classifier.predict(X_Train)
    y_pred_test = classifier.predict(X_test)

    return y_pred_train, y_pred_test

def train_lf_model(emotion):  
    model = fasttext.load_model(FT_MODEL_PATH)
    
    text_train, y_train = get_texts_and_labels(BASE_PATH, emotion, "train")
    text_test, y_test = get_texts_and_labels(BASE_PATH, emotion, "test")

    X_train = get_avg_vector(text_train, model)
    X_test = get_avg_vector(text_test, model)

    X_train, X_test = scale(X_train, X_test)
    
    classifier = LogisticRegression(multi_class='ovr')
    y_pred_train, y_pred_test = predict(classifier, X_train, X_test, y_train)
    
    return classifier, X_test, y_test, y_pred_test

    
hate_lr_model, X_test_hate, y_test_hate, y_pred_test_hate = train_lf_model("hate")
offensive_lr_model, X_test_offensive, y_test_offensive, y_pred_test_offensive = train_lf_model("offensive")

In [177]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score


def classification_metrics_lr(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    
    print(f'Accuracy LR :{acc}')
    print(f'Confusion Matrix LR: \n{cm}')
    print(f'Classification Report LR: \n{report}')
    
def classification_metrics_lstm(model, dataloader):
    model = model.to(device)
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Processing", unit="batch"):
            inputs = batch['text'].float().to(device)
            lengths = batch['length']
            outputs = model(inputs, lengths)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions


def calculate_accuracy_lstm(predictions, labels):
    correct = sum(p == l for p, l in zip(predictions, labels))
    accuracy = correct / len(labels) * 100
    return accuracy

## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_1 (2 pkt.)

In [178]:
classification_metrics_lr(y_test_hate, y_pred_test_hate)


train_dataloader, embedding_dim, class_count = get_dataloader(FT_MODEL_PATH, BASE_PATH, 'hate', 'test')
predictions = classification_metrics_lstm(hate_lstm_model, train_dataloader)

accuracy_lstm = calculate_accuracy_lstm(predictions, y_test_hate)
print(f"Accuracy LSTM: {accuracy_lstm}%")

Accuracy LR :0.5576398362892224
Confusion Matrix LR: 
[[1522  175]
 [1122  113]]
Classification Report LR: 
              precision    recall  f1-score   support

           0       0.58      0.90      0.70      1697
           1       0.39      0.09      0.15      1235

    accuracy                           0.56      2932
   macro avg       0.48      0.49      0.42      2932
weighted avg       0.50      0.56      0.47      2932



Processing: 100%|██████████| 92/92 [00:00<00:00, 180.81batch/s]

Accuracy LSTM: 57.094133697135064%


## Ewaluacja modeli na danych testowych dla zjawiska ZJAWISKO_2 (2 pkt.)

In [180]:
classification_metrics_lr(y_test_offensive, y_pred_test_offensive)


train_dataloader, embedding_dim, class_count = get_dataloader(FT_MODEL_PATH, BASE_PATH, 'offensive', 'test')
predictions = classification_metrics_lstm(offensive_lstm_model, train_dataloader)

accuracy_lstm = calculate_accuracy_lstm(predictions, y_test_offensive)
print(f"Accuracy LSTM: {accuracy_lstm}%")

Accuracy LR :0.75
Confusion Matrix LR: 
[[597  23]
 [192  48]]
Classification Report LR: 
              precision    recall  f1-score   support

           0       0.76      0.96      0.85       620
           1       0.68      0.20      0.31       240

    accuracy                           0.75       860
   macro avg       0.72      0.58      0.58       860
weighted avg       0.73      0.75      0.70       860



Processing: 100%|██████████| 27/27 [00:00<00:00, 166.31batch/s]

Accuracy LSTM: 66.97674418604652%
